## Preparing for Data Scraping

Public Data Source: https://www.seoul.go.kr/coronaV/coronaStatus.do

In [455]:
import pandas as pd
import numpy as np

In [456]:
import requests

In [457]:
url = "https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw=1"
#url = f"{url}&columns%5B0%5D%5Bdata%5D=0&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=true&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=1&columns%5B1%5D%5Bname%5D=&columns%5B1%5D%5Bsearchable%5D=true&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D=2&columns%5B2%5D%5Bname%5D=&columns%5B2%5D%5Bsearchable%5D=true&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B3%5D%5Bdata%5D=3&columns%5B3%5D%5Bname%5D=&columns%5B3%5D%5Bsearchable%5D=true&columns%5B3%5D%5Borderable%5D=true&columns%5B3%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B3%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B4%5D%5Bdata%5D=4&columns%5B4%5D%5Bname%5D=&columns%5B4%5D%5Bsearchable%5D=true&columns%5B4%5D%5Borderable%5D=true&columns%5B4%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B4%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B5%5D%5Bdata%5D=5&columns%5B5%5D%5Bname%5D=&columns%5B5%5D%5Bsearchable%5D=true&columns%5B5%5D%5Borderable%5D=true&columns%5B5%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B5%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B6%5D%5Bdata%5D=6&columns%5B6%5D%5Bname%5D=&columns%5B6%5D%5Bsearchable%5D=true&columns%5B6%5D%5Borderable%5D=true&columns%5B6%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B6%5D%5Bsearch%5D%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=0&order%5B0%5D%5Bdir%5D=desc"
url = f"{url}&start=0&length=100"
#"&search%5Bvalue%5D=&search%5Bregex%5D=true&_=1628930508535"
url

'https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw=1&start=0&length=100'

In [458]:
response = requests.get(url)
data_json = response.json()
# pd.DataFrame(data_json['data'])


In [459]:
records_total = data_json['recordsTotal']
records_total

40870

In [460]:
end_page = round(records_total/ 100) + 1
end_page                                        # used in get_multi_page_list function

410

In [461]:
data = data_json["data"]

In [462]:
pd.DataFrame(data)

,0,1,2,3,4,5,6
0,<p class='corona19_no'>70870</p>,222111,2021-08-13,기타,-,감염경로 조사중,<b class=''>-</b>
1,<p class='corona19_no'>70869</p>,221232,2021-08-13,기타,-,감염경로 조사중,<b class=''>-</b>
2,<p class='corona19_no'>70868</p>,221356,2021-08-13,성북구,-,감염경로 조사중,<b class=''>-</b>
3,<p class='corona19_no'>70867</p>,221710,2021-08-13,동대문구,-,감염경로 조사중,<b class=''>-</b>
4,<p class='corona19_no'>70866</p>,220502,2021-08-13,타시도,-,기타 확진자 접촉,<b class=''>-</b>
...,...,...,...,...,...,...,...
95,<p class='corona19_no'>70775</p>,221585,2021-08-13,은평구,-,감염경로 조사중,<b class=''>-</b>
96,<p class='corona19_no'>70774</p>,221083,2021-08-13,동작구,-,기타 확진자 접촉,<b class=''>-</b>
97,<p class='corona19_no'>70773</p>,221319,2021-08-13,동작구,-,수도권 소재 실내체육시설 관련,<b class=''>-</b>
98,<p class='corona19_no'>70772</p>,221342,2021-08-13,동작구,-,기타 확진자 접촉,<b class=''>-</b>


## Function for scraping one pages

In [463]:
def get_seoul_covid19_100(page_no):
    start_no = (page_no - 1) * 100
    url = f"https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw={page_no}"
    url = f"{url}&start={start_no}&length=100"
    response = requests.get(url)
    data_json = response.json()
    return data_json

In [464]:
# data available
get_seoul_covid19_100(1)

{'draw': 1,
 'recordsTotal': 40870,
 'recordsFiltered': 40870,
 'data': [["<p class='corona19_no'>70870</p>",
   '222111',
   '2021-08-13',
   '기타',
   '-',
   '감염경로 조사중',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>70869</p>",
   '221232',
   '2021-08-13',
   '기타',
   '-',
   '감염경로 조사중',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>70868</p>",
   '221356',
   '2021-08-13',
   '성북구',
   '-',
   '감염경로 조사중',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>70867</p>",
   '221710',
   '2021-08-13',
   '동대문구',
   '-',
   '감염경로 조사중',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>70866</p>",
   '220502',
   '2021-08-13',
   '타시도',
   '-',
   '기타 확진자 접촉',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>70865</p>",
   '220262',
   '2021-08-13',
   '동대문구',
   '-',
   '감염경로 조사중',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>70864</p>",
   '221253',
   '2021-08-13',
   '타시도',
   '-',
   '기타 확진자 접촉',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>70863</p>",

In [465]:
# data unavailable
get_seoul_covid19_100(-1)

{'draw': -1, 'recordsTotal': 40870, 'recordsFiltered': 40870, 'data': []}

## Python code for combining all pages

In [466]:
# to prevent network demage
import time
# tqdm: process check
from tqdm import trange

In [467]:
def get_multi_page_list(start_page, end_page):
    page_list = []
    for page_no in trange(start_page, end_page + 1):
        one_page = get_seoul_covid19_100(page_no)
        # break when page is empty
        if len(one_page['data']) > 0:
            one_page = pd.DataFrame(one_page['data'])
            page_list.append(one_page)
        else:
            return page_list
    return page_list
        
    


In [468]:
end_page

410

In [469]:
# Test Code
start_page = 1
end_page = round(records_total/ 100) + 1
# end_page = 205
page_list = get_multi_page_list(start_page, end_page)
page_list[:1]

100%|█████████▉| 409/410 [08:34<00:01,  1.26s/it]


[                                   0       1           2     3  4  \
 0   <p class='corona19_no'>70870</p>  222111  2021-08-13    기타  -   
 1   <p class='corona19_no'>70869</p>  221232  2021-08-13    기타  -   
 2   <p class='corona19_no'>70868</p>  221356  2021-08-13   성북구  -   
 3   <p class='corona19_no'>70867</p>  221710  2021-08-13  동대문구  -   
 4   <p class='corona19_no'>70866</p>  220502  2021-08-13   타시도  -   
 ..                               ...     ...         ...   ... ..   
 95  <p class='corona19_no'>70775</p>  221585  2021-08-13   은평구  -   
 96  <p class='corona19_no'>70774</p>  221083  2021-08-13   동작구  -   
 97  <p class='corona19_no'>70773</p>  221319  2021-08-13   동작구  -   
 98  <p class='corona19_no'>70772</p>  221342  2021-08-13   동작구  -   
 99  <p class='corona19_no'>70771</p>  221590  2021-08-13   은평구  -   
 
                    5                  6  
 0           감염경로 조사중  <b class=''>-</b>  
 1           감염경로 조사중  <b class=''>-</b>  
 2           감염경로 조사중  <b cla

In [470]:
df_all = pd.concat(page_list)
df_all.shape

(40870, 7)

In [471]:
cols = ['Number','Patient','Confimed Date','Area','Travel History','Transmission Route','Status']

In [472]:
df_all.columns = cols

In [473]:
df_all.head()

,Number,Patient,Confimed Date,Area,Travel History,Transmission Route,Status
0,<p class='corona19_no'>70870</p>,222111,2021-08-13,기타,-,감염경로 조사중,<b class=''>-</b>
1,<p class='corona19_no'>70869</p>,221232,2021-08-13,기타,-,감염경로 조사중,<b class=''>-</b>
2,<p class='corona19_no'>70868</p>,221356,2021-08-13,성북구,-,감염경로 조사중,<b class=''>-</b>
3,<p class='corona19_no'>70867</p>,221710,2021-08-13,동대문구,-,감염경로 조사중,<b class=''>-</b>
4,<p class='corona19_no'>70866</p>,220502,2021-08-13,타시도,-,기타 확진자 접촉,<b class=''>-</b>


In [474]:
df_all.shape

(40870, 7)

## Data Preprocessing

### Number and Status

In [475]:
import re
def extract_number(num_string):
    if type(num_string) == str:
        num_string = num_string.replace("corona19", "")
        num = re.sub("[^0-9]", "", num_string)
        num = int(num)
        return num
    else:
        return num_string

In [476]:
num_string = "<p class='corona19_no'>7625</p>"
extract_number(num_string)

7625

In [477]:
df_all['Number'] = df_all["Number"].map(extract_number)

In [478]:
df_all.head()

,Number,Patient,Confimed Date,Area,Travel History,Transmission Route,Status
0,70870,222111,2021-08-13,기타,-,감염경로 조사중,<b class=''>-</b>
1,70869,221232,2021-08-13,기타,-,감염경로 조사중,<b class=''>-</b>
2,70868,221356,2021-08-13,성북구,-,감염경로 조사중,<b class=''>-</b>
3,70867,221710,2021-08-13,동대문구,-,감염경로 조사중,<b class=''>-</b>
4,70866,220502,2021-08-13,타시도,-,기타 확진자 접촉,<b class=''>-</b>


In [479]:
def extract_hangeul(origin_text):
    subtract_text = re.sub("[^가-힣]", "", origin_text)
    return subtract_text

In [480]:
extract_hangeul("<b class='status1'>퇴원</b>")

'퇴원'

In [481]:
extract_hangeul("<b class='status2'>사망</b>")

'사망'

In [482]:
extract_hangeul("<b class=''>-</b>")

''

In [483]:
df_all["Status"].value_counts()

<b class='status1'>퇴원</b>    32131
<b class='status1'>-</b>      4991
<b class=''>-</b>             3531
<b class='status2'>사망</b>      117
<b class='status2'>-</b>       100
Name: Status, dtype: int64

In [484]:
df_all.loc[df_all["Status"].str.contains("퇴원"), "Status"] = "Recovered"
df_all.loc[df_all["Status"].str.contains("사망"), "Status"] = "Death"
df_all.loc[~df_all["Status"].str.contains("Recovered|Death"), "Status"] = "Quarantined"
df_all

,Number,Patient,Confimed Date,Area,Travel History,Transmission Route,Status
0,70870,222111,2021-08-13,기타,-,감염경로 조사중,Quarantined
1,70869,221232,2021-08-13,기타,-,감염경로 조사중,Quarantined
2,70868,221356,2021-08-13,성북구,-,감염경로 조사중,Quarantined
3,70867,221710,2021-08-13,동대문구,-,감염경로 조사중,Quarantined
4,70866,220502,2021-08-13,타시도,-,기타 확진자 접촉,Quarantined
...,...,...,...,...,...,...,...
65,30005,95804,2021-03-14,송파구,-,타시도 확진자 접촉,Recovered
66,30004,95889,2021-03-14,종로구,-,감염경로 조사중,Recovered
67,30003,95799,2021-03-14,중구,-,중구 소재 주점/음식점 관련,Recovered
68,30002,95872,2021-03-14,종로구,-,기타 확진자 접촉,Recovered


In [485]:
df_all.loc[df_all["Transmission Route"].str.contains("조사중"), "Transmission Route"] = "Under Inspection"
df_all.loc[df_all["Transmission Route"].str.contains("접촉|관련"), "Transmission Route"] = "Known"
df_all.loc[df_all["Transmission Route"].str.contains("해외"), "Transmission Route"] = "International"
df_all.loc[~df_all["Transmission Route"].str.contains("Under Inspection|Known|International"), "Transmission Route"] = "Unknown"
df_all

,Number,Patient,Confimed Date,Area,Travel History,Transmission Route,Status
0,70870,222111,2021-08-13,기타,-,Under Inspection,Quarantined
1,70869,221232,2021-08-13,기타,-,Under Inspection,Quarantined
2,70868,221356,2021-08-13,성북구,-,Under Inspection,Quarantined
3,70867,221710,2021-08-13,동대문구,-,Under Inspection,Quarantined
4,70866,220502,2021-08-13,타시도,-,Known,Quarantined
...,...,...,...,...,...,...,...
65,30005,95804,2021-03-14,송파구,-,Known,Recovered
66,30004,95889,2021-03-14,종로구,-,Under Inspection,Recovered
67,30003,95799,2021-03-14,중구,-,Known,Recovered
68,30002,95872,2021-03-14,종로구,-,Known,Recovered


In [486]:
df_all.loc[df_all["Area"].str.contains("종로구"), "Area"] = "Jongno-gu"
df_all.loc[df_all["Area"].str.contains("중구"), "Area"] = "Jung-gu"
df_all.loc[df_all["Area"].str.contains("용산구"), "Area"] = "Yongsan-gu"
df_all.loc[df_all["Area"].str.contains("성동구"), "Area"] = "Seongdong-gu"
df_all.loc[df_all["Area"].str.contains("광진구"), "Area"] = "Gwangjin-gu"
df_all.loc[df_all["Area"].str.contains("동대문구"), "Area"] = "Dongdaemun-gu"
df_all.loc[df_all["Area"].str.contains("중랑구"), "Area"] = "Jungnang-gu"
df_all.loc[df_all["Area"].str.contains("성북구"), "Area"] = "Seongbuk-gu"
df_all.loc[df_all["Area"].str.contains("강북구"), "Area"] = "angbuk-gu"
df_all.loc[df_all["Area"].str.contains("도봉구"), "Area"] = "Dobong-gu"
df_all.loc[df_all["Area"].str.contains("노원구"), "Area"] = "Nowon-gu"
df_all.loc[df_all["Area"].str.contains("은평구"), "Area"] = "Eunpyeong-gu"
df_all.loc[df_all["Area"].str.contains("서대문구"), "Area"] = "Seodaemun-gu"
df_all.loc[df_all["Area"].str.contains("마포구"), "Area"] = "Mapo-gu"
df_all.loc[df_all["Area"].str.contains("양천구"), "Area"] = "Yangcheon-gu"
df_all.loc[df_all["Area"].str.contains("강서구"), "Area"] = "Gangseo-gu"
df_all.loc[df_all["Area"].str.contains("구로구"), "Area"] = "Guro-gu"
df_all.loc[df_all["Area"].str.contains("금천구"), "Area"] = "Geumcheon-gu"
df_all.loc[df_all["Area"].str.contains("영등포구"), "Area"] = "Yeongdeungpo-gu"
df_all.loc[df_all["Area"].str.contains("동작구"), "Area"] = "Dongjak-gu"
df_all.loc[df_all["Area"].str.contains("관악구"), "Area"] = "Gwanak-gu"
df_all.loc[df_all["Area"].str.contains("서초구"), "Area"] = "Seocho-gu"
df_all.loc[df_all["Area"].str.contains("강남구"), "Area"] = "Gangnam-gu"
df_all.loc[df_all["Area"].str.contains("송파구"), "Area"] = "Songpa-gu"
df_all.loc[df_all["Area"].str.contains("강동구"), "Area"] = "Gangdong-gu"
df_all.loc[df_all["Area"].str.contains("타시도|기타"), "Area"] = "Another City"

In [487]:
file_name = 'covid_19_recent_data.csv'
df_all.to_csv(file_name, index=False)

In [488]:
pd.read_csv(file_name)

,Number,Patient,Confimed Date,Area,Travel History,Transmission Route,Status
0,70870,222111,2021-08-13,Another City,-,Under Inspection,Quarantined
1,70869,221232,2021-08-13,Another City,-,Under Inspection,Quarantined
2,70868,221356,2021-08-13,Seongbuk-gu,-,Under Inspection,Quarantined
3,70867,221710,2021-08-13,Dongdaemun-gu,-,Under Inspection,Quarantined
4,70866,220502,2021-08-13,Another City,-,Known,Quarantined
...,...,...,...,...,...,...,...
40865,30005,95804,2021-03-14,Songpa-gu,-,Known,Recovered
40866,30004,95889,2021-03-14,Jongno-gu,-,Under Inspection,Recovered
40867,30003,95799,2021-03-14,Jung-gu,-,Known,Recovered
40868,30002,95872,2021-03-14,Jongno-gu,-,Known,Recovered
